# Lectura 37: LazyFrame - Manipulación y selección

Debemos tener en cuenta que estas operaciones funcionan de la misma forma a como lo hacen con los DataFrame con algunas excepciones. Como ya vimos una gran variedad de estas funciones en la sección anterior del curso en esta ocasión solo mostraremos algunos ejemplos.

In [ ]:
import polars as pl

vuelos_lf = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
)

## `botton_k`

Retorna los k elementos más pequeños del DataFrame. Le podemos indicar la cantidad de elementos que deseamos mostrar y por cual columna ordenar para obtener el resultado.

In [ ]:
vuelos_lf.bottom_k(20, by='DEPARTURE_TIME', descending=True, nulls_last=True).collect()

## `drop` 

Elimina la(s) columna(s) seleccionada(s).

In [ ]:
vuelos_lf.drop('YEAR', 'MONTH', 'DAY').collect()

## Filtrar un DataFrame

In [ ]:
from polars import col

(
    vuelos_lf
    .filter((col('MONTH') > 6) & (col('DEPARTURE_DELAY') < 0))
    .select(
        col('MONTH'),
        col('DEPARTURE_DELAY')
    ).collect()
)

## `with_context`

Esta función agrega un contexto externo al gráfico de cálculo. Esto permite que las expresiones también accedan a columnas de DataFrames que no forman parte de este.

In [ ]:
tarjetas_lz = pl.LazyFrame({'tarjeta': ['bronce', 'plata', 'oro']})

descuentos_lz = pl.LazyFrame({'descuento': [10, 20, 30]})

tarjetas_lz.with_context(descuentos_lz).select(
    col('tarjeta'),
    col('descuento')
).collect()

## `slice`

Con esta función podemos obtener una porción o lo que en inglés se conoce como un slice de un DataFrame.

In [ ]:
vuelos_lf.slice(1,10).collect()

## `sort`

Esta función ordena el LazyFrame por la(s) columna(s) proporcionadas.

In [ ]:
vuelos_lf.sort('DEPARTURE_DELAY', descending=True, nulls_last=True).collect()

## `gather_every`

Toma cada enésima fila del LazyFrame y regresa un nuevo LazyFrame. El primer parámetro estable cada cuantas filas queremos tomar la siguiente fila y el parámetro offset establece desde que posición queremos iniciar a tomar las filas.

In [ ]:
vuelos_lf.gather_every(5, 100).collect()

## `shift`

Esta función cambia los valores según el número dado de índices. En la práctica lo podemos ver como un desplazamiento de las filas del LazyFrame.

In [ ]:
vuelos_lf.shift(3).collect()

In [ ]:
vuelos_lf.shift(3, fill_value='prueba').collect()

## `inspect`

Esta función inspecciona un nodo en el gráfico de cálculo. Con ella podemos imprimir el valor que evalúa este nodo en el gráfico de cálculo.

In [ ]:
query_inspect = (
    vuelos_lf.with_columns(AIR_TIME_HRS=(col('AIR_TIME') / 60))
    .inspect()
    .select(
        col('AIRLINE'),
        col('AIR_TIME_HRS')
    )
)

In [ ]:
query_inspect.collect()

In [ ]:
query_inspect.explain()

Hasta aquí llegaremos en este recorrido de las funciones que nos permiten manipular y seleccionar elementos de un LazyFrame. En de que ustedes necesiten ver la lista competa de todas las funciones que están disponibles pueden dirigirse a la documentación de Polars.

[LazyFrame - Manipulación y selección](https://docs.pola.rs/py-polars/html/reference/lazyframe/modify_select.html)